# Homework: $-KRC$ Filter
Simon Burkhardt, FHNW/ISE, 12.05.2021

In [1]:
%matplotlib notebook
import sympy as sp
import numpy as np
import math
import scipy as sc
import scipy.signal
import matplotlib.pyplot as plt
import hanspitools as ht
def niceT(T,s): # This is a shortcut for a functionality I use very often
    return ht.mani.numden(lambda p: ht.mani.nicepoly(p,s),T)
def keeper(f,v):
    return sp.limit(f/v,v,sp.oo)*v

<img src="KRC_Schematic.png" alt="schematic" width="800"/>

<img src="KRC_SFG.png" alt="SFG" width="900"/>

In [28]:
import sympy as sp
Delta = sp.symbols('Delta')
L1,L2,L3,L4 = sp.symbols('L1,L2,L3,L4')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
A,K,G1,Y3,G2,G3,Y2,Y1,G4 = sp.symbols('A,K,G1,Y3,G2,G3,Y2,Y1,G4')

loops = [(L1, -A*G2*G3*G4/(Y1*Y2*Y3)), (L2, -A*G4/(K*Y3)), (L3, G2**2/(Y1*Y2)), (L4, G4**2/(Y2*Y3))]
determinant = [(Delta, -L1 + L2*L3 - L2 - L3 - L4 + 1)]
denominator = [(T_den, Delta)]

P1,D1 = sp.symbols('P1,D1')
paths = [(P1, -A*G1*G2*G4/(Y1*Y2*Y3)), (D1, 1)]
numerator = [(T_num, D1*P1)]

transfer_function = [(T_io, T_num/T_den)]
T=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(T)

A*G1*G2*G4*K/(A*G2**2*G4 - A*G2*G3*G4*K - A*G4*Y1*Y2 + G2**2*K*Y3 + G4**2*K*Y1 - K*Y1*Y2*Y3)

In [29]:
# Substitute the DP-Impedances 
# Substitute to only use equal components
G,s,C1,C2,C=sp.symbols('G,s,C1,C2,C')
eqY1 = [(Y1,G1+G2+G3+s*C1)]
eqY2 = [(Y2,G2+G4+s*C2)]
eqY3 = [(Y3, G4,)]

Tio = T.subs(eqY1).subs(eqY2).subs(eqY3)
Tio = Tio.subs([(G1,G),(G2,G),(G3,G),(G4,G)])
Tio = Tio.subs([(C1, C),(C2,C)]).simplify()
Tio = sp.limit(Tio,A,sp.oo)
niceT(Tio,s)

G**2*K/(-C**2*s**2 - 5*C*G*s - G**2*(K + 5))

This is a 2nd order low pass filter, because there is an $s^2$ in the denominator and no $s$ in the numerator.

In [30]:
# bring the denominator in the correct form
gain = 1/(C**2)
Tio_g = (Tio / gain).simplify()
niceT(Tio_g,s)

-C**2*G**2*K/(C**2*s**2 + 5*C*G*s + G**2*(K + 5))

In [24]:
# rewrite by hand and divide the denominator by C^2
# to bring it into the form s^2 + wp/qp*s + wp^2*s
Tio_g = (G**2*K)/(s**2 + 5*G/C*s + G**2/C**2*(K+5))
Tio_g

G**2*K/(s**2 + 5*G*s/C + G**2*(K + 5)/C**2)

In [6]:
# extract the denominator and its coefficients
num,den = sp.fraction(Tio_g)
coeffs = sp.Poly(den,s).coeffs()
# extract wp^2
wp_2 = coeffs[2]
wp = sp.sqrt(wp_2)
wp

sqrt((G**2*K + 5*G**2)/C**2)

In [25]:
# rewrite the square root
wp = G/C * sp.sqrt(K+5)
wp

G*sqrt(K + 5)/C

In [26]:
# with known wp: calculate qp next
wp_qp = coeffs[1]
qp = wp/wp_qp
qp

sqrt(K + 5)/5

---

### b) Design a low-pass fitler with $f_0=2kHz$, $Q=5$ and $0dB$ DC gain

In [27]:
# the full transfer function
w_p,Q=sp.symbols('omega_p,Q')
T_filt = (C**2*G**2*K)/(s**2 + w_p/Q*s + w_p**2)
niceT(T_filt,s)

C**2*G**2*K/(omega_p**2 + s**2 + omega_p*s/Q)

In [10]:
# 
w3db = 2*sp.pi*2e3;
Qp = 5;
# 1. condition at DC:
# must be ==0 !
display((C**2*G**2*K)/(w_p**2))
eqDC = sp.Eq(1, (C**2*G**2*K)/(w_p**2))

C**2*G**2*K/omega_p**2

In [11]:
# 2. condition 
# wp must be 2pi 2kHz
display(wp)
eqWp = sp.Eq(w3db, wp)

G*sqrt(K + 5)/C

In [12]:
# make assumptions about real world components
sp.Q.positive(G)
sp.Q.positive(C)
sp.Q.positive(K)
sp.Q.real(G)
sp.Q.real(C)
sp.Q.real(K)
# pick a capacitor
eqC = sp.Eq(C, 2e-6)
# set omega_p
eqWp = sp.Eq(wp, w3db)
lsg = sp.solve([eqWp, eqWp, eqDC, eqC])
lsg

[]

If DC gain = 1 is considered, no solution is found.

In [13]:
# ignore the DC-gain constraint
eqWp = sp.Eq(wp, w3db)
lsg = sp.solve([eqWp, eqWp, eqC])
lsg

[{C: 2.00000000000000e-6, G: 0.0112397035696652/sqrt(0.2*K + 1)}]

In [14]:
lsg[0][G]

0.0112397035696652/sqrt(0.2*K + 1)

In order for G to be a non-complex value, K must be $K>5$.

Let us assume $K=620$ to find a close solution.

In [19]:
eqK = sp.Eq(K,620)
lsg = sp.solve([eqWp, eqWp, eqK, eqC])
lsg

[{C: 2.00000000000000e-6, K: 620.000000000000, G: 0.00100530964914873}]

In [20]:
R = 1/lsg[0][G].evalf()
R

994.718394324346

These are $1k\Omega$ resistors and the DC gain is now:

In [33]:
Cc=lsg[0][C].evalf()
Gg=lsg[0][G].evalf()
Kk=lsg[0][K].evalf()
w_pp=w3db

dcgain = (Cc**2*Gg**2*Kk)/(w_pp**2).evalf()
display(dcgain)

1.58720000000000e-23

this is wrong, Hanspi's solution gives $-\frac{124}{125}$